__Tilldelar VNUM 1-3 i varje avdelning en score för vinst__

In [1]:
# Common imports
import numpy as np
import os
import pandas as pd

In [2]:
# Necessary Sklearn objects used in the analysis
from sklearn.metrics import roc_curve, auc
from sklearn.ensemble import RandomForestClassifier 
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn import preprocessing

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Imputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import FeatureUnion

In [3]:
# DataPiplene är en pythonfil som innehåller den egenkostruerade klassen DataFrameSelector
from DataPipeline import DataFrameSelector
from sklearn.externals import joblib

/Users/ulf/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [7]:
import sklearn

In [8]:
print(sklearn.__version__)

0.21.2


__Läser in filen som ska köras__

In [4]:
df1 = pd.read_excel('TestDataV75TillUffeMP20200516.xlsx')

In [5]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82 entries, 0 to 81
Data columns (total 36 columns):
Datum          82 non-null datetime64[ns]
Arstid         82 non-null int64
Distans        82 non-null int64
Startsatt      82 non-null int64
Lopp           82 non-null int64
Plac           82 non-null int64
Hast           82 non-null int64
V75PROC        82 non-null float64
V_ODDS         82 non-null float64
GRUPP          82 non-null int64
VLP            82 non-null float64
VNUM           82 non-null int64
SVLP           82 non-null float64
VSVLP          82 non-null float64
VPN_SUM        82 non-null float64
VPN_SUM_ORD    82 non-null int64
VPK_SUM        82 non-null float64
VPK_SUM_ORD    82 non-null int64
VLPB           82 non-null float64
SVLPB          82 non-null float64
VSVLPB         82 non-null float64
E_P            82 non-null float64
E_P_Num        82 non-null int64
E_N            82 non-null float64
E_R            82 non-null float64
E_U            82 non-null float64
G_

In [6]:
df1.SP_R.value_counts()

0    62
3     7
1     7
2     6
Name: SP_R, dtype: int64

In [7]:
df1.R_R.value_counts()

0    61
3     7
2     7
1     7
Name: R_R, dtype: int64

In [8]:
df1[['SP_R', 'R_R']].corr()

,SP_R,R_R
SP_R,1.000000,0.377739
R_R,0.377739,1.000000


In [5]:
df1 = df1[df1.VNUM.isin([1,2,3])]    

In [6]:
df1 = df1.sort_values(['Lopp','Hast'], ascending = ['True','False']).reset_index()

In [7]:
# Importera model som ska användas
rf = joblib.load('Travmodel_top1.pkl')

In [8]:
# Import av datapipeline som ska användas
full_pipeline = joblib.load('Pipeline_top1.pkl')

__Kör modellen och plockar listan med högsta score__

In [9]:
features = full_pipeline.fit_transform(df1)

In [10]:
predict = rf.predict_proba(features)

In [11]:
# Gör om scorad array till en lista - multidimensionell
stack = predict.tolist()

In [12]:
# Sedan plockar vi den scorade sannolikheten att vara vinnare - 1
# Transformerar denna sannolikhet till en dataframe
last = []
for x in stack:
    last.append(x[1])
scored = {'Score':last}
# Konverterar till Dataframe
df_scored = pd.DataFrame.from_dict(scored)

In [13]:
len(df_scored)

21

In [14]:
df_predict2 = df1.merge(df_scored,  right_index = True, left_index = True) 

In [15]:
df1_out = df_predict2[['Lopp','Hast','Score','E_P_Num','VNUM']].sort_values(['Lopp','Score'], ascending = [True,False])
df1_out

,Lopp,Hast,Score,E_P_Num,VNUM
0,1,1,0.240380,6,1
1,1,2,0.133905,6,2
2,1,9,0.133871,6,3
3,2,1,0.338609,5,1
4,2,3,0.293646,5,3
5,2,4,0.181899,5,2
8,3,12,0.309728,2,1
7,3,11,0.304332,2,2
6,3,4,0.209688,2,3
9,4,10,0.352207,1,1


In [16]:
df1_out.sort_values('Score', ascending = False)

,Lopp,Hast,Score,E_P_Num,VNUM
9,4,10,0.352207,1,1
3,2,1,0.338609,5,1
12,5,2,0.333941,4,2
15,6,4,0.326788,3,1
13,5,4,0.311559,4,1
16,6,5,0.310711,3,2
8,3,12,0.309728,2,1
7,3,11,0.304332,2,2
4,2,3,0.293646,5,3
17,6,7,0.292756,3,3


__Export till excel__

In [17]:
df1_out.to_excel('Test20200516.xlsx', index = False)